### Elevation Download

With this notebook, you can download elevation data for whole world. Also, you can define elevation data if data includes value bigger than 0. With this method you can eliminate data on the sea(ocean).

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
# If you are running Firefox on a system with no display, make sure you use headless mode.
# We use Selenium to export information from map
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get('https://www.earthenv.org/DEM')

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
dem_page=BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
maps_element=dem_page.map

In [1]:
#Convert the Bs4 element to list

In [ ]:
for index,img in enumerate(maps_element,0):
    imgs.append(img)
del imgs[0:2]
del imgs[-2:]
# whole world
len(imgs)

In [7]:
# main folder of your download
target_folder='./data/'
if not os.path.isdir(target_folder):
    os.mkdir(target_folder)
# .tar.gz file location
zip_file_folder=target_folder+'elevation_zip/'
if not os.path.isdir(zip_file_folder):
    os.mkdir(zip_file_folder)
# Unzip file location
elevation_files_folder=target_folder+'elevation_files/'
if not os.path.isdir(elevation_files_folder):
    os.mkdir(elevation_files_folder)


In [ ]:
for img in imgs:
    url=img['href']
    img_name=img['href'].split('/')[-1]
    img_path=zip_file_folder+img_name
    #print(img_path)
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(img_path, 'wb') as f:
            f.write(response.raw.read())
    
    #unzip the file
    file_name=img['href'].split('/')[-1].split('.')[0]
    file_path=elevation_files_folder+file_name
    #print(file_path)
    shutil.unpack_archive(img_path,file_path,'gztar')
    

#### If all value in elevation file(.bil) bigger than 0, we define and translate to 'tif' format


In [1]:
import glob
import os
from osgeo import gdal,ogr,osr
import numpy as np

In [4]:
imgs_folder='./data/elevation_files/*/'
search_criteria = "*.bil"
glob_criteria = os.path.join(imgs_folder, search_criteria)
paths = glob.glob(glob_criteria)

In [6]:
target_folder='./data/'
output_folder=target_folder+'elevation_tif/'
if not os.path.isdir(output_folder):
    os.mkdir(output_folder)

In [7]:
%%time
for i in paths:
    img=gdal.Open(i)
    img_array=img.ReadAsArray()
    if np.all(img_array==0):
        continue
    else:
        tif_name=os.path.basename(i).split('.')[0]
        output_path=output_folder+tif_name+'.tif'
        #print(output_path)
        
        
        s_srs = img.GetProjectionRef()
        osng = osr.SpatialReference ()
        osng.SetFromUserInput ( s_srs )
        geo_t = img.GetGeoTransform ()
        x_size = img.RasterXSize # Raster xsize
        y_size = img.RasterYSize # Raster ysize
        
        #create gdal memory file to convert image to EPSG:4326        
        mem_drv = gdal.GetDriverByName( 'MEM' )
        dest = mem_drv.Create('', x_size,y_size, 1, gdal.GDT_Int16)
        dest.SetGeoTransform( geo_t )
        dest.SetProjection ( osng.ExportToWkt() )
        dest.GetRasterBand(1).SetNoDataValue(-32768)
        dest.GetRasterBand(1).WriteArray(img_array)
        
        gdal.Warp(output_path, dest, format = 'GTiff', dstSRS = 'EPSG:4326 ')   
        dst_ds=None
        mem_drv=None
        
        
        

./data/elevation_tif/EarthEnv-DEM90_N40W005.tif
./data/elevation_tif/EarthEnv-DEM90_N40W010.tif
CPU times: user 1.17 s, sys: 133 ms, total: 1.31 s
Wall time: 1.3 s
